# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data_group=data.groupby(['CustomerID','ProductName']).agg({'Quantity':np.sum})
# data_group.Quantity = data_group.Quantity.sum()
data_group.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
data_pivot=pd.pivot_table(data_group, values='Quantity', index=['ProductName'],columns=['CustomerID'],fill_value=0)
data_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
squareform(pdist(data_pivot, 'euclidean'))

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'euclidean'))), 
                         index=data_pivot.columns, columns=data_pivot.columns)

similarities = distances[33].sort_values(ascending=False)[0:5]
similarities

CustomerID
33      1.000000
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
recommendations = data_group.copy()

list_series=[recommendations.loc[name] for name, score in dict(similarities).items()]

df_step5 = pd.concat(list_series)
df_step5.head()

,Quantity
ProductName,
Apricots - Dried,1
Assorted Desserts,1
Bandage - Flexible Neon,1
"Bar Mix - Pina Colada, 355 Ml",1
"Beans - Kidney, Canned",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
prod_quantity=df_step5.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
prod_quantity.head()

,Quantity
ProductName,
"Pepper - Black, Whole",4
Butter - Unsalted,3
Grouper - Fresh,3
"Veal - Inside, Choice",3
Veal - Sweetbread,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
data_pivot[data_pivot[33]==0].merge(prod_quantity, left_on='ProductName', right_on='ProductName').sort_values(by='Quantity',ascending=False)[[33,'Quantity']].head()

,33,Quantity
ProductName,,
Soup - Campbells Bean Medley,0,3
Butter - Unsalted,0,3
Lamb - Ground,0,2
"Mushrooms - Black, Dried",0,2
Chocolate - Dark,0,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
dictionary_step8={}
list_uniqueCustomers=[k for k,i in dict(similarities).items()]
list_uniqueCustomers

def generate_product(metric):
    distances = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, metric))), 
                         index=data_pivot.columns, columns=data_pivot.columns)
    dictionary_step8={}
    list_uniqueCustomers=[k for k,i in dict(similarities).items()]
    for i in list_uniqueCustomers:
        similarities_2 = distances[i].sort_values(ascending=False)[0:5]
        recommendations_2 = data_group.copy()
        list_series_2=[recommendations_2.loc[name] for name, score in dict(similarities_2).items()]
        df_step5_2 = pd.concat(list_series_2)
        prod_quantity_2=df_step5_2.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
        dictionary_step8[i]=(data_pivot[data_pivot[i]==0].merge(prod_quantity_2, left_on='ProductName', right_on='ProductName')[[i,'Quantity']].sort_values(by='Quantity',ascending=False).head()).index
    return dictionary_step8

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [12]:
pd.DataFrame.from_dict(generate_product('euclidean'))

,33,264,3535,3317,2503
0,Soup - Campbells Bean Medley,Bread - Italian Roll With Herbs,Butter - Unsalted,Knife Plastic - White,"Pepper - Black, Whole"
1,Butter - Unsalted,Wine - Two Oceans Cabernet,Ecolab - Solid Fusion,Yoghurt Tubes,Soup - Campbells Bean Medley
2,Lamb - Ground,"Veal - Inside, Choice",Knife Plastic - White,Sausage - Breakfast,Wine - Blue Nun Qualitatswein
3,"Mushrooms - Black, Dried",Durian Fruit,Crush - Cream Soda,Wine - Blue Nun Qualitatswein,Soupfoamcont12oz 112con
4,Chocolate - Dark,Beef - Top Sirloin,Muffin - Carrot Individual Wrap,Bar - Granola Trail Mix Fruit Nut,Sherry - Dry


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [13]:
list_metric=['correlation','cityblock','cosine','jaccard']
for i in list_metric:
    display(pd.DataFrame.from_dict(generate_product(i)))

,33,264,3535,3317,2503
0,Muffin - Zero Transfat,"Water - Mineral, Natural",Flavouring - Orange,Garlic,"Water, Tap"
1,Banana Turning,Wine - Toasted Head,Hersey Shakes,"Lamb - Whole, Fresh",Cheese - Cottage Cheese
2,Crush - Cream Soda,Snapple - Iced Tea Peach,Wine - Ruffino Chianti,Cheese - Taleggio D.o.p.,Lettuce - Frisee
3,Veal - Osso Bucco,Garbag Bags - Black,Appetizer - Mushroom Tart,Bananas,Wine - Sogrape Mateus Rose
4,Onions - Cippolini,Hersey Shakes,Broom - Corn,Soupcontfoam16oz 116con,Pastry - Raisin Muffin - Mini


,33,264,3535,3317,2503
0,Bay Leaf,Bread - Italian Roll With Herbs,Butter - Unsalted,Ketchup - Tomato,"Pepper - Black, Whole"
1,Chocolate - Dark,Wine - Two Oceans Cabernet,Ecolab - Solid Fusion,Knife Plastic - White,Scallops 60/80 Iqf
2,Wine - Charddonnay Errazuriz,"Veal - Inside, Choice",Knife Plastic - White,Anchovy Paste - 56 G Tube,Muffin - Carrot Individual Wrap
3,Sauerkraut,Durian Fruit,Crush - Cream Soda,Ocean Spray - Ruby Red,Soup - Campbells Bean Medley
4,V8 - Berry Blend,Beef - Top Sirloin,Muffin - Carrot Individual Wrap,Foam Dinner Plate,Scampi Tail


,33,264,3535,3317,2503
0,Knife Plastic - White,Pickerel - Fillets,Juice - Happy Planet,Garlic,Cheese - Cottage Cheese
1,Muffin - Zero Transfat,"Water - Mineral, Natural",Flavouring - Orange,"Lamb - Whole, Fresh",Pastry - Raisin Muffin - Mini
2,Banana Turning,Snapple - Iced Tea Peach,Wine - Ruffino Chianti,Cheese - Taleggio D.o.p.,Pail For Lid 1537
3,Crush - Cream Soda,Wine - Ej Gallo Sierra Valley,Hersey Shakes,Bananas,"Water, Tap"
4,Chips Potato All Dressed - 43g,French Pastry - Mini Chocolate,Fuji Apples,Soupcontfoam16oz 116con,Beef - Texas Style Burger


,33,264,3535,3317,2503
0,Loquat,Pasta - Cheese / Spinach Bauletti,"Appetizer - Mini Egg Roll, Shrimp","Spoon - Soup, Plastic",Muffin Batt - Choc Chk
1,Wine - Toasted Head,"Lamb - Pieces, Diced",Soup Knorr Chili With Beans,Appetizer - Sausage Rolls,Rice - Long Grain
2,Scampi Tail,Bread - Italian Corn Meal Poly,Sobe - Tropical Energy,Sobe - Tropical Energy,Pie Filling - Cherry
3,Ketchup - Tomato,General Purpose Trigger,"Lemonade - Natural, 591 Ml",Tomatoes Tear Drop,Tea - Jasmin Green
4,Tea - Jasmin Green,Beer - Original Organic Lager,Muffin - Carrot Individual Wrap,"Juice - Cranberry, 341 Ml",Halibut - Fletches
